In [1]:
# s = pd.Series(len_status)
# print(s.describe())
# len_status[-1]
# dic
import tqdm

In [1]:
# build hugface datasets
from datasets import load_dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")
# processin funtion
def lowercase_condition(examples):
    examples["paragraphs"] = [x.lower() for x in examples["paragraphs"]]
    return  examples

def filter_nones(x):
    return x["condition"] is not None
# filter only provide __liter__ __next__ so the return type is filter
# map batch== True require the example be list type
# Remark: did before turn into datasets
# def filter_sentence_length(example):
#     print()
#     for _ in example["paragraphs"]:

#     add_on = list(filter(lambda x: len(x) > 5 ,example["paragraphs"]))
#     example["paragraphs"] = add_on 
#     return example

def tokenize_and_split(examples):
    return tokenizer(
        examples["paragraphs"],
        truncation=True,
        max_length=512,
        return_overflowing_tokens=True,
    )
def tokenize_function(examples):
    lower = [x.lower() for x in examples["paragraphs"]]
    result = tokenizer(lower)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

DatasetDict({
    train: Dataset({
        features: ['paragraphs', 'topic', 'ID'],
        num_rows: 1578212
    })
    test: Dataset({
        features: ['paragraphs', 'topic', 'ID'],
        num_rows: 175357
    })
})

mean       75.771333
std        51.978543
min        20.000000
25%        38.000000
50%        61.000000
75%        97.000000
max       435.000000

In [2]:
hklii_dataset = load_dataset("json", data_files="/home/xijia/nlp/data_prepare/data/HKLII_zh.json")
# print(hklii_dataset)
#hklii_dataset = hklii_dataset.shuffle(seed=42) # not need for this step
# hklii_dataset = hklii_dataset.map(
#     lowercase_condition, batched=True,num_proc = 16
# )
hklii_dataset = hklii_dataset["train"].train_test_split(test_size=0.1)
print(hklii_dataset)

Using custom data configuration default-a712c258280d2958
Reusing dataset json (/home/huijie/.cache/huggingface/datasets/json/default-a712c258280d2958/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['paragraphs', 'topic', 'ID'],
        num_rows: 1458006
    })
    test: Dataset({
        features: ['paragraphs', 'topic', 'ID'],
        num_rows: 162001
    })
})


In [4]:
print(hklii_dataset["train"][:3])
stat = [len(inp) for inp in hklii_dataset["train"]["paragraphs"]]
import pandas as pd
s = pd.Series(stat)
print(s.describe())

{'paragraphs': ['12.  PW1-PW5是在反毒行動中扮作顧客的卧底警員；', '申請人： 由法律援助署委派何柏生馬華潤律師行轉聘大律師孫錦熹代表。', '14'], 'topic': ['HKDC', 'HKCA', 'HKCFI'], 'ID': ['2016_605.json', '2012_1.json', '2002_1338.json']}
count    1.458006e+06
mean     7.819433e+01
std      9.874830e+01
min      1.000000e+00
25%      1.500000e+01
50%      4.400000e+01
75%      1.080000e+02
max      6.492000e+03
dtype: float64


In [4]:
result = tokenize_function(hklii_dataset["train"][1:3])
print(result)
print(tokenizer)

{'input_ids': [[101, 113, 123, 114, 4302, 2433, 4316, 2770, 2433, 1651, 2586, 2881, 5975, 1651, 8224, 6566, 3449, 2881, 10032, 1985, 2380, 2074, 6812, 2081, 1968, 109, 31367, 5975, 9661, 9648, 113, 6392, 1732, 4107, 6824, 114, 1483, 102], [101, 113, 1651, 1866, 3246, 5724, 114, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'word_ids': [[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, None], [None, 0, 1, 2, 3, 4, 5, None]]}
PreTrainedTokenizerFast(name_or_path='bert-base-multilingual-uncased', vocab_size=105879, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': 

In [5]:
tokenized_datasets = hklii_dataset.map(
    tokenize_function, batched=True, remove_columns=["ID","topic","paragraphs"],num_proc = 16
)
tokenized_datasets

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (601 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (635 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (610 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (684 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 1458006
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 162001
    })
})

In [6]:
tokenized_samples = tokenized_datasets["train"][:20]
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 1088'


In [7]:
chunk_size = 512
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 64'


In [8]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result


In [9]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True,num_proc = 32)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 198802
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 22025
    })
})

In [8]:
print(tokenizer.decode(lm_datasets["train"][1]["input_ids"]))

print(tokenizer.decode(lm_datasets["train"][1]["labels"]))

邀 請 他 人 加 入 三 合 會 。 [SEP] [CLS] 14. 裁 判 官 不 給 予 pw1 證 詞 比 重, 鑑 於 裁 判 官 對 證 人 有 耳 聞 目 睹 的 優 勢, 提 出 的 理 由 也 合 理, 本 席 於 考 慮 時 將 pw1 的 證 詞 拼 棄 。 [SEP] [CLS] 10. 10 月 2 日 ， 申 請 人 向 教 育 統 籌 局 投 訴 學 校 處 理 19 / 2 / 04 事 件 不 當 ， 但 沒 有 結 果 。 [SEP] [CLS] d8 的 控 罪 [SEP] [CLS] 判 案 書 [SEP] [CLS] 6. 本 案 申 請 人 的 上 訴 無 理 據 支 持 。 本 庭 駁 回 上 訴 ， 不 作 出 訟 費 命 令 。 [SEP] [CLS] 其 後 ， 警 員 搜 查 那 黑 色 環 保 袋 ， 發 現 一 袋 涉 案 「 k 仔 」 。 根 據 警 方 資 料 ， 這 些 「 k 仔 」 的 零 售 價 約 為 二 萬 九 千 元 。 [SEP] [CLS] 25. 上 述 2004 年 10 月 27 日 已 付 糧 單 顯 示 鋪 磚 面 積 為 343. 7 平 方 米 ( 見 文 件 夾 第 86 - 87 頁 ) 。 上 述 2004 年 11 月 27 日 已 付 糧 單 顯 示 鋪 磚 面 積 為 1, 188. 9 平 方 米 、 完 成 5 個 沙 井 蓋 及 提 供 3 工 代 工 ( 見 文 件 夾 第 88 頁 ) 。 上 述 2004 年 12 月 27 日 已 付 糧 單 顯 示 鋪 磚 面 積 為 3, 886. 95 平 方 米 、 完 成 13 個 沙 井 蓋 及 提 供 6 工 代 工 ( 見 文 件 夾 第 89 - 90 頁 ) 。 [SEP] [CLS] 2. 經 審 訊 後 ， 本 席 於 本 年 6 月 21 日 ， 頒 下 判 案 書 ( 下 簡 稱 [UNK] 判 案 書 [UNK] ) ， 判 周 女 士 敗 訴 ， 撤 銷 了 她 的 申 索 。 按 此 結 果 ， 本 席 同 時 作 出 暫 時 命 令 ， 周 女 士 須 支 付 署 方 因 訴 訟 引 起 的 訟 費 ， 包 括 大 律 師 費 用 。 除 非 雙 方 協 議 ， 訟 費 由 法

In [10]:
sents = ["However, D2 was forced to move out from PW9's premises around the midnight of 6/7 November 2016.","D2 simply had no time to give advance notice.","Though the defendant had indicated his consent earlier, how D2 could ensure he would be allowed entry.","The defendant liked glasses and was ordering glasses for his collection, the burglary of the optical company committed by D2 probably was not a mere coincidence, but to secure his entry into the defendant’s premises."] 
model_inputs = tokenizer(text_pair = sents)
model_inputs

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [9]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [10]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [MASK] [MASK] 及 [MASK] 政 [MASK] 訟 案 2007 年 第 78 號 [SEP] [CLS] 判 案 書 日 期 [MASK] 2006 年 12 月 11 日 [SEP] [CLS] 46. 本 席 必 [MASK] 明 確 指 出 [MASK] 父 母 子 女 ， 夫 teorije ， 或 兄 弟 姊 妹 在 家 事 上 之 安 [MASK] 協 議 ， 除 [MASK] 在 事 前 特 別 聲 [MASK] 可 [MASK] 互 雙 追 討 ， 清 楚 顯 明 雙 方 有 建 [MASK] 法 律 關 係 之 意 願 ， 令 該 安 排 協 議 [MASK] 生 法 [MASK] 後 果 ， 否 則 在 法 律 上 沒 有 任 何 約 束 力 。 [SEP] [CLS] 25. 至 於 上 訴 人 שימש 將 連 女 士 的 信 件 交 objekata [MASK] 方 證 人 們 ， 是 不 分 [MASK] 紅 皂 白 ， 抑 或 是 托 詞 ？ 上 訴 人 指 他 回 覆 管 理 處 不 會 將 信 件 交 予 [MASK] 人 [MASK] 會 直 接 交 回 公 司 。 即 使 是 托 辭 不 交 [MASK] [MASK] 是 有 心 為 難 [MASK] 行 為 並 不 合 理 。 潘 大 律 師 指 行 [MASK] 不 [MASK] 理 不 代 表 不 可 信 。 [SEP] [CLS] [UNK] [SEP] [CLS] 32. 經 [MASK] 訊 之 後 ， 被 裁 定 第 十 一 、 第 十 三 、 第 [MASK] 四 及 第 十 [MASK] 項 控 罪槐 cochrane 四 項 控 罪 罪 名 成 立 ， [MASK] 四 項 控 罪 分 別 是 第 十 一 [MASK] 第 十 五 項 控 罪 〈 以 三 合 會 社 團 成 [MASK] 身 分 行 事 〉 、 第 十 三 及 [MASK] 十 四 項 控 [MASK] [MASK] 邀 請 他 人 成 為 [MASK] halbinsel 會 成 [MASK] 〉 [MASK] 所 以 嚴 格 來 說 ， 四 項 控 罪 都 是 與 [MASK] 合 會 有 關 之 餘 [MASK] 就 並 不 是 一 般 最 

In [13]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id

    return default_data_collator(features)

In [14]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "zlucia/custom-legalbert"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at zlucia/custom-legalbert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(lm_datasets["train"]) // batch_size
model_name = "hklii"

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)


/home/huijie/miniconda3/envs/hugface/lib/python3.8/site-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [22]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
)

In [22]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


trainer.train()


eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

NameError: name 'trainer' is not defined

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from huggingface_hub import create_repo

repo_url = create_repo(name="HKLII", repo_type="dataset")
repo_url

'https://huggingface.co/datasets/phjhk/HKLII'

In [16]:
hklii_dataset.save_to_disk("/home/huijie/legal/huggface/data_prepare/data")

Flattening the indices:   0%|          | 0/1579 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/176 [00:00<?, ?ba/s]

In [18]:
from huggingface_hub import Repository

repo = Repository(local_dir="/home/huijie/legal/huggface/data_prepare/HKLII-online", clone_from="https://huggingface.co/datasets/phjhk/HKLII")
!cp datasets-issues-with-comments.jsonl github-issues/

Cloning https://huggingface.co/datasets/phjhk/HKLII into local empty directory.


cp: cannot stat 'datasets-issues-with-comments.jsonl': No such file or directory


## play with the tokenlizer and try to slove the unk issue


In [5]:

hklii_dataset = hklii_dataset.map(
    lowercase_condition, batched=True, remove_columns=["ID","topic"],num_proc = 16
)
hklii_dataset

DatasetDict({
    train: Dataset({
        features: ['paragraphs'],
        num_rows: 1578212
    })
    test: Dataset({
        features: ['paragraphs'],
        num_rows: 175357
    })
})

In [11]:
example = hklii_dataset["train"]['paragraphs'][13]
print(example)
print( " ".join(tokenizer.tokenize(example)))

2.  經審訊後，本席於本年6月21日，頒下判案書(下簡稱“判案書”)，判周女士敗訴，撤銷了她的申索。按此結果，本席同時作出暫時命令，周女士須支付署方因訴訟引起的訟費，包括大律師費用。除非雙方協議，訟費由法庭評定。雙方可以於14天内提出申請要求更改訟費命令，否則命令將自動作實。
2 . 經 審 訊 後 ， 本 席 於 本 年 6 月 21 日 ， 頒 下 判 案 書 ( 下 簡 稱 [UNK] 判 案 書 [UNK] ) ， 判 周 女 士 敗 訴 ， 撤 銷 了 她 的 申 索 。 按 此 結 果 ， 本 席 同 時 作 出 暫 時 命 令 ， 周 女 士 須 支 付 署 方 因 訴 訟 引 起 的 訟 費 ， 包 括 大 律 師 費 用 。 除 非 雙 方 協 議 ， 訟 費 由 法 庭 評 定 。 雙 方 可 以 於 14 天 内 提 出 申 請 要 求 更 改 訟 費 命 令 ， 否 則 命 令 將 自 動 作 實 。


now we know that there are some method to remove unk
1. remove the , ' in the 
2. add words : may improve the performance and 

In [8]:
import string
import re
#example = "(ii)   If so, whether it was reasonable for P to continue its action against D1 after receiving the Joint Report and/or after the withdrawal of contribution notice against D1 by D2?"

# print(new_str)
# print( " ".join(tokenizer.tokenize(new_str)))


def clean_data(example):
    tmp = re.sub(r'^(\d+).', '', example)
    tmp = re.sub(r'^\((ix|iv|v?i{0,3})\)', '', tmp)
    tmp = re.sub(r'^\((\d+)\)', '', tmp)
    new_str = tmp.translate(str.maketrans('','',string.punctuation)).replace("’s",'').replace("s’",'s')
    return new_str

for i in hklii_dataset_ch["train"]['paragraphs'][15:20]:
    clean = clean_data(i)
    print(i)
    print(clean)
    print( " ".join(tokenizer.tokenize(clean)))
    print("___________________")


總結
總結
總 結
___________________
1.  被告擅闖民居，進行盜竊。本席採納一般的入獄3年，為判刑起點。
  被告擅闖民居，進行盜竊。本席採納一般的入獄3年，為判刑起點。
被 告 擅 闖 民 居 ， 進 行 盜 竊 。 本 席 採 納 一 般 的 入 獄 3 年 ， 為 判 刑 起 點 。
___________________
27.  答辯人若已準備向申請人提出申索,倘若法院信納答辯人的理據並判決答辯人勝數,答辯人亦理應獲判其損失的賠償, 所以不會因收樓令而造成不能彌補的事實。
  答辯人若已準備向申請人提出申索倘若法院信納答辯人的理據並判決答辯人勝數答辯人亦理應獲判其損失的賠償 所以不會因收樓令而造成不能彌補的事實。
答 辯 人 若 已 準 備 向 申 請 人 提 出 申 索 倘 若 法 院 信 納 答 辯 人 的 理 據 並 判 決 答 辯 人 勝 數 答 辯 人 亦 理 應 獲 判 其 損 失 的 賠 償 所 以 不 會 因 收 樓 令 而 造 成 不 能 彌 補 的 事 實 。
___________________
6. 本案申請人的上訴無理據支持。本庭駁回上訴，不作出訟費命令。
 本案申請人的上訴無理據支持。本庭駁回上訴，不作出訟費命令。
本 案 申 請 人 的 上 訴 無 理 據 支 持 。 本 庭 駁 回 上 訴 ， 不 作 出 訟 費 命 令 。
___________________
頒下判案書日期 ： 2001年9月28日
頒下判案書日期 ： 2001年9月28日
頒 下 判 案 書 日 期 ： 2001 年 9 月 28 日
___________________


### build chinese and chinese-english datasets togeter


In [3]:
hklii_dataset_ch = load_dataset("json", data_files="/home/xijia/nlp/data_prepare/data/HKLII_zh.json")
hklii_dataset_ch = hklii_dataset_ch["train"]
print(hklii_dataset_ch)

Using custom data configuration default-a712c258280d2958
Reusing dataset json (/home/huijie/.cache/huggingface/datasets/json/default-a712c258280d2958/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['paragraphs', 'topic', 'ID'],
    num_rows: 1620007
})


In [4]:
hklii_dataset_en = load_dataset("json", data_files="/home/huijie/legal/huggface/data_prepare/HKLII-online/HKLII.json")
hklii_dataset_en = hklii_dataset_en["train"]
print(hklii_dataset_en)

Using custom data configuration default-59ce93af8ba84fa0
Reusing dataset json (/home/huijie/.cache/huggingface/datasets/json/default-59ce93af8ba84fa0/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['paragraphs', 'topic', 'ID'],
    num_rows: 1753569
})


In [5]:
# filter the length < 5 
def clean_data(example):
    tmp = re.sub(r'^(\d+).', '', example)
    tmp = re.sub(r'^\((ix|iv|v?i{0,3})\)', '', tmp)
    tmp = re.sub(r'^\((\d+)\)', '', tmp)
    new_str = tmp.translate(str.maketrans('','',string.punctuation)).replace("’s",'').replace("s’",'s')
    return new_str

# date prepared for training
def tokenize_function(examples):
    lower = [clean_data(x.lower()) for x in examples["paragraphs"]]
    lower = filter(lambda x : len(x) >7, lower )
    result = tokenizer(lower)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [6]:
from datasets import concatenate_datasets

assert hklii_dataset_en.features.type == hklii_dataset_ch.features.type
hklii_dataset = concatenate_datasets([hklii_dataset_en, hklii_dataset_ch])
print(hklii_dataset)



Dataset({
    features: ['paragraphs', 'topic', 'ID'],
    num_rows: 3373576
})


In [7]:
hklii_dataset = hklii_dataset.train_test_split(test_size=0.1)

tokenized_datasets = hklii_dataset.map(
    tokenize_function, batched=True, remove_columns=["ID","topic","paragraphs"],num_proc = 8
)
print(tokenized_datasets)

In [ ]:
hklii_dataset.save_to_disk("/home/huijie/legal/huggface/data_prepare/HKLII_all")

In [ ]:
lower = [len(x) for x in tokenized_datasets["train"]["input_ids"][:100] if len(x) > 7]

In [20]:
stat = [len(inp) for inp in hklii_dataset["train"]["paragraphs"]]
import pandas as pd
s = pd.Series(stat)
print(s.describe())

count    3.036218e+06
mean     2.645207e+02
std      3.038398e+02
min      1.000000e+00
25%      4.800000e+01
50%      1.750000e+02
75%      3.750000e+02
max      5.867500e+04
dtype: float64


In [1]:
from transformers import BertTokenizer, BertForPreTraining
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = BertForPreTraining.from_pretrained("bert-base-multilingual-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

prediction_logits = outputs.prediction_logits
seq_relationship_logits = outputs.seq_relationship_logits


Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
print(**input)
print(prediction_logits.shape)
print(seq_relationship_logits.shape)

AttributeError: 'function' object has no attribute 'shape'